<a href="https://colab.research.google.com/github/SudhA-2k25/PythonBio/blob/main/colab/DiffSBDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DiffSBDD: Structure-based Drug Design with Equivariant Diffusion Models

[**[Paper]**](https://arxiv.org/abs/2210.13695)
[**[Code]**](https://github.com/arneschneuing/DiffSBDD)

Make sure to select `Runtime` -> `Change runtime type` -> `GPU` before you run the script.

<img src="https://raw.githubusercontent.com/arneschneuing/DiffSBDD/main/img/overview.png" height=250>

In [3]:
#@title  Install condacolab (the kernel will be restarted, after that you can execute the remaining cells)
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...


In [1]:
#@title Install dependencies (this will take about 5-10 minutes)
%cd /content

import os

commands = [
    "pip install torch==2.0.1 --extra-index-url https://download.pytorch.org/whl/cu118",
    "pip install pytorch-lightning==1.8.4",
    "pip install wandb==0.13.1",
    "pip install rdkit==2022.3.3",
    "pip install biopython==1.79",
    "pip install imageio==2.21.2",
    "pip install scipy==1.7.3",
    "pip install pyg-lib torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html",
    "pip install networkx==2.8.6",
    "pip install py3Dmol==1.8.1",
    "conda install openbabel -c conda-forge",
    "git clone https://github.com/arneschneuing/DiffSBDD.git",
    "mkdir -p /content/DiffSBDD/checkpoints",
    "wget -P /content/DiffSBDD/checkpoints https://zenodo.org/record/8183747/files/moad_fullatom_cond.ckpt",
    "wget -P /content/DiffSBDD/checkpoints https://zenodo.org/record/8183747/files/moad_fullatom_joint.ckpt",
]

errors = {}

if not os.path.isfile("/content/READY"):
  for cmd in commands:
    # os.system(cmd)
    with os.popen(cmd) as f:
      out = f.read()
      status = f.close()

    if status is not None:
      errors[cmd] = out
      print(f"\n\nAn error occurred while running '{cmd}'\n")
      print("Status:\t", status)
      print("Message:\t", out)

if len(errors) == 0:
  os.system("touch /content/READY")

/content


## Choose target PDB

In [2]:
from google.colab import files
from google.colab import output
output.enable_custom_widget_manager()
import os.path
from pathlib import Path
import urllib
import os

input_dir = Path("/content/input_pdbs/")
output_dir = Path("/content/output_sdfs/")
input_dir.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)

target = "upload structure" #@param ["example (3rfm)", "upload structure"]

if target == "example (3rfm)":
  pdbfile = Path(input_dir, '3rfm.pdb')
  urllib.request.urlretrieve('http://files.rcsb.org/download/3rfm.pdb', pdbfile)

elif target == "upload structure":
  uploaded = files.upload()
  fn = list(uploaded.keys())[0]
  pdbfile = Path(input_dir, fn)
  Path(fn).rename(pdbfile)

Saving 7qn5.pdb to 7qn5.pdb


## Define binding pocket

You can choose between two options to define the binding pocket:
1. **list of residues:** provide a list where each residue is specified as `<chain_id>:<res_id>`, e.g, `A:1 A:2 A:3 A:4 A:5 A:6 A:7`
2. **reference ligand:** if the uploaded PDB structure contains a reference ligand in the target pocket, you can specify its location as `<chain_id>:<res_id>` and the pocket will be extracted automatically

In [3]:
#@title { run: "auto" }
import ipywidgets as widgets

#@markdown **Note:** This cell is an interactive widget and the values will be updated automatically every time you change them. You do not need to execute the cell again. If you do, the default values will be reinserted.

pocket_definition = "reference ligand" #@param ["list of residues", "reference ligand"]

if pocket_definition == "list of residues":
  print('pocket_residues:')
  w = widgets.Text(value='A:9 A:59 A:60 A:62 A:63 A:64 A:66 A:67 A:80 A:81 A:84 A:85 A:88 A:167 A:168 A:169 A:170 A:172 A:174 A:177 A:181 A:246 A:249 A:250 A:252 A:253 A:256 A:265 A:267 A:270 A:271 A:273 A:274 A:275 A:277 A:278')
  pocket_flag = "--resi_list"
elif pocket_definition == "reference ligand":
  print('reference_ligand:')
  w = widgets.Text(value='a:1')
  pocket_flag = "--ref_ligand"

display(w)

reference_ligand:


Text(value='a:1')

## Settings

Notes:
- `timesteps < 500` is an experimental feature
- `resamplings` and `jump_length` only pertain to the inpainting model

In [4]:
#@markdown ## Sampling
n_samples = 10 #@param {type:"slider", min:1, max:100, step:1}
ligand_nodes = 40 #@param {type:"integer"}

model = "Conditional model (Binding MOAD)" #@param ["Conditional model (Binding MOAD)", "Inpainting model (Binding MOAD)"]
checkpoint = Path('/content', 'DiffSBDD', 'checkpoints', 'moad_fullatom_cond.ckpt') if model == "Conditional model (Binding MOAD)" else Path('DiffSBDD', 'checkpoints', 'moad_fullatom_joint.ckpt')

timesteps = 100 #@param {type:"slider", min:1, max:500, step:1}

#@markdown  ## Inpainting parameters
resamplings = 1 #@param {type:"integer"}
jump_length = 1 #@param {type:"integer"}

#@markdown  ## Post-processing
keep_all_fragments = False #@param {type:"boolean"}
keep_all_fragments = "--all_frags" if keep_all_fragments else ""
sanitize = True #@param {type:"boolean"}
sanitize = "--sanitize" if sanitize else ""
relax = True #@param {type:"boolean"}
relax = "--relax" if relax else ""

In [5]:
# Install the necessary packages
# Ensure protobuf version is compatible
!pip install protobuf==3.20.3 torch

# Set the environment variable
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Navigate to the correct directory
%cd /content/DiffSBDD

import argparse
from pathlib import Path
import torch
import utils
from lightning_modules import LigandPocketDDPM

# Parameters
pdbfile = '/content/input_pdbs/7qn5.pdb'  # Replace with actual uploaded PDB file
checkpoint = '/content/DiffSBDD/checkpoints/moad_fullatom_cond.ckpt'
output_dir = '/content/output_sdfs'
n_samples = 1
ligand_nodes = 30
pocket_flag = '--ref_ligand'
sanitize = '--sanitize'
relax = '--relax'
resamplings = 3
jump_length = 1
timesteps = 200

# Reference ligand definition (chain and residue number)
# Example: If the reference ligand is at chain 'a', residue '1', specify it like this
pocket = 'a:1'  # Replace this with the actual chain and residue of your ligand

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the model
model = LigandPocketDDPM.load_from_checkpoint(checkpoint, map_location=device)
model = model.to(device)

# Ligand nodes
num_nodes_lig = torch.ones(n_samples, dtype=int) * ligand_nodes

# Generate ligands using the reference ligand
try:
    molecules = model.generate_ligands(
        pdbfile, n_samples, resi_list=None, ref_ligand=pocket,  # Using reference ligand instead of list of residues
        num_nodes_lig=num_nodes_lig, sanitize=sanitize,
        largest_frag=not relax, relax_iter=(200 if relax else 0),
        resamplings=resamplings, jump_length=jump_length,
        timesteps=timesteps
    )

    # Save SDF files
    pdb_id = Path(pdbfile).stem
    utils.write_sdf_file(Path(output_dir, f'{pdb_id}_mol.sdf'), molecules)
    print("Ligands generated and saved successfully!")

except Exception as e:
    print(f"Error generating ligands: {e}")


/content/DiffSBDD


/usr/local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Entropy of n_nodes: H[N] 9.149419784545898
Ligands generated and saved successfully!


/usr/local/lib/python3.10/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/content/DiffSBDD/analysis/molecule_builder.py:224: UserWarning: Maximum number of FF iterations reached. Returning molecule after 200 relaxation steps.
  warnings.warn(f'Maximum number of FF iterations reached. '


In [21]:
# Install protobuf version 3.20.x
!pip install protobuf==3.20.3 torch

# Set the environment variable to use pure Python implementation
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Restart runtime to ensure changes take effect
print("Please restart the runtime after running this cell.")


Please restart the runtime after running this cell.


In [6]:
# Install necessary dependencies
!pip install py3Dmol

import sys
from pathlib import Path
import py3Dmol

# Ensure pdbfile is a Path object
pdbfile = Path('/content/input_pdbs/7qn5.pdb')  # Example path, update as needed
output_dir = '/content/output_sdfs'

# Create a 3Dmol viewer
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)

# Add the PDB structure to the viewer
pdb_content = open(pdbfile, 'r').read()
view.addModel(pdb_content, 'pdb')
view.setStyle({'model': -1}, {'cartoon': {'color': 'lime'}})

# Add generated ligands (SDF) as models
sdf_file = Path(output_dir, f"{pdbfile.stem}_mol.sdf")  # Use Path object for file paths
view.addModelsAsFrames(open(sdf_file, 'r').read())

# Set style for the ligands
view.setStyle({'model': -1}, {'stick': {}})

# Zoom and animate the view
view.zoomTo({'model': -1})
view.zoom(0.5)

# Rotate the view if needed (only for example target)
target = "example (3rfm)"  # Update based on your target condition
if target == "example (3rfm)":
    view.rotate(90, 'y')

# Animate the viewer
view.animate({'loop': "forward", 'interval': 1000})
view.show()

# Save the structure as a new PDB file in the output directory
structure_pdb = Path(output_dir, "structure.pdb")
with open(structure_pdb, 'w') as f:
    f.write(pdb_content)

print(f"Structure saved as {structure_pdb}")


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Structure saved as /content/output_sdfs/structure.pdb


In [10]:
#@title Download .sdf file
files.download(Path(output_dir, f"{pdbfile.stem}_mol.sdf"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>